In [1]:
# -*- coding: utf-8 -*-
import fitz  # PyMUPDF
from pylatexenc.latexwalker import LatexWalker, LatexEnvironmentNode
from pylatexenc.latex2text import LatexNodes2Text


def decodePdf(path):
    doc = fitz.open(path)
    for page in doc:
        text = page.getText()
    return text


def decodeTex(path):
    doc = " ".join(open(path, "r").readlines()).encode("ISO-8859-1").decode("utf-8")
    return LatexNodes2Text().latex_to_text(doc)


print(decodePdf("true_or_false_question.pdf"))
print(decodeTex("answer_1_true.tex"))


Considere o programa, Pyhton 3, que se segue.
Ignore a vari´avel seed e a fun¸c˜ao
pseudo_random_integer, que se destinam exclusivamente `a gera¸c˜ao de n´umeros pseudo-
aleat´orios.
seed = 1116111
def pseudo random integer ( min int ,
max int ) :
global seed
seed = (16807∗ seed ) % 2147483647
return int ( min int + ( max int − min int ) ∗ seed / 2147483646)
h = [ ]
for p in range (19624):
h . append ( pseudo random integer (750 ,
1750))
Acrescente a este programa o c´odigo que lhe permita indicar se as aﬁma¸c˜oes seguintes s˜ao
verdadeiras ou falsas.
Indique se ´e veradeiro ou falso.


 
 O elemento da lista , no índice , é 1690.
 
 
